# Tumbling Quantification

## Impots

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits import mplot3d
from scipy.optimize import curve_fit

## Import Video and Extract Boundaries

In [ ]:
def extractBoundaries(fileName, blur):

    ContoursSeries = []

    # Extract video information
    cap = cv2.VideoCapture(fileName)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Loop through all frames
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:

            # Convert color to gray, remove timer/scale bar, erode, binarize, save
            grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            kernel = np.ones((blur,blur), np.uint8)
            blurImg = cv2.GaussianBlur(grayImg,(blur,blur),0)
            erodeImg = cv2.erode(blurImg, kernel, iterations=1)
            ret, binImg = cv2.threshold(erodeImg,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Display original and binarized with contours
            binImgColor = cv2.cvtColor(binImg,cv2.COLOR_GRAY2RGB)
            contours, hierarchy = cv2.findContours(binImg, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            ContoursSeries.append(contours)
            cv2.drawContours(binImgColor, contours, -1, (0,255,0), 0)
            combined = np.concatenate((frame, binImgColor), axis = 1)
            cv2.imshow('frame',combined)
            if cv2.waitKey(50) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    return ContoursSeries

## Rotated Ellipse Equation

In [ ]:
def rotatedEllipse(X, a, b, theta):
    x, y = X
    result = ((1/b * np.sin(theta))**2 + (1/a * np.cos(theta))**2) * x**2 + 2 * ((1/a)**2 - (1/b)**2) * np.sin(theta) * np.cos(theta) * x * y + ((1/b * np.cos(theta))**2 + (1/a * np.sin(theta))**2) * y**2
    return result

## Calculate Params Over All Frames and Rate of Change

In [ ]:
def ellipseFit(ContoursSeries):
    xCents = []
    yCents = []
    aVals = []
    bVals= []
    thetaVals = []
    xValsAll = []
    yValsAll = []
    for i in range(len(ContoursSeries)):
        boundaries = ContoursSeries[i]
        boundaries.sort(key=len, reverse=True)
        # Cell boundary always second longest after overall outline of the entire image
        if len(boundaries) <= 1:
            xCents.append(np.nan)
            yCents.append(np.nan)
            aVals.append(np.nan)
            bVals.append(np.nan)
            thetaVals.append(np.nan)
        else:
            cellBound = boundaries[1]
            xvals = np.asarray([pair[0][0] for pair in cellBound])
            yvals = np.asarray([pair[0][1] for pair in cellBound])
            xValsAll.append(xvals)
            yValsAll.append(yvals)
            # Calculate mean to shift values to "origen"
            xcent = np.mean(xvals)
            ycent = np.mean(yvals)
            ones = np.zeros(len(xvals)) + 1
            X = (xvals - xcent, yvals - ycent)
            xCents.append(xcent)
            yCents.append(ycent)
            params, cov = curve_fit(rotatedEllipse, X, ones, bounds=(0, [np.inf, np.inf, np.pi]), method = 'trf')
            aVals.append(params[0])
            bVals.append(params[1])
            thetaVals.append(params[2])
    return xCents, yCents, aVals, bVals, thetaVals, xValsAll, yValsAll

In [ ]:
def findDistancesCoords(xCoords, yCoords):
    diffs = []
    x1 = xCoords[0]
    y1 = yCoords[0]
    for i in range(len(xCoords) - 1):
        x2 = xCoords[i + 1]
        y2 = yCoords[i + 1]
        if x2 != np.nan:
            diff = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            diffs.append(diff)
            x1 = x2
            y1 = y2
    return diffs

In [ ]:
def avLinVel(xCoords, yCoords, timeGap):
    diffs = findDistancesCoords(xCoords, yCoords)
    return np.nanmean(np.absolute(diffs)) / timeGap

In [ ]:
def fixThetas(aVals, bVals, thetaVals):
    thetaNew = []
    halfCirc = np.pi / 2
    for i in range(len(thetaVals)):
        if aVals[i] >= bVals[i]:
            thetaNew.append(thetaVals[i])
        else: 
            # Case where minor axis in second quad
            if thetaVals[i] >= halfCirc:
                thetaNew.append(thetaVals[i] - halfCirc)
            # Case where minor axis first quad
            else: 
                thetaNew.append(thetaVals[i] + halfCirc)
    return thetaNew   
                

In [ ]:
def findDifference(vals):
    diffs = []
    first = vals[0]
    for i in range(len(vals) - 1):
        second = vals[i + 1]
        if second != np.nan:
            # Assumption here is that rotation from frame to frame take min path 
            diff = min(abs(second - first), abs((second - np.pi) - first), abs(second - (np.pi - first)))
            diffs.append(diff)
            first = second
    return diffs

In [ ]:
def avAngularVel(vals, timeGap):
    diffs = findDifference(vals)
    return np.nanmean(np.absolute(diffs)) / timeGap

## Visualize the Fit for the Ellipses

In [ ]:
def visualizeEllipse(fileName, ContoursSeries, xCents, yCents, aVals, bVals, thetaVals):
    
    cap = cv2.VideoCapture(fileName)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    i = 0

    # Make object to save the video
#     saveName = 'Modified.avi'  # change the file name if needed
#     imgSize = (height, width)
#     frame_per_second = fps
#     writer = cv2.VideoWriter(saveName, cv2.VideoWriter_fourcc('M','J','P','G'), frame_per_second, imgSize)
    
    # Loop through all frames
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            
            # Convert color to gray, remove timer/scale bar, erode, binarize, save
            grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            kernel = np.ones((13,13), np.uint8)
            blurImg = cv2.GaussianBlur(grayImg,(13,13),0)
            erodeImg = cv2.erode(blurImg, kernel, iterations=1)
            ret, binImg = cv2.threshold(erodeImg,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Display original and binarized with contours + ellipse fit
            binImgColor = cv2.cvtColor(binImg,cv2.COLOR_GRAY2RGB)
            cv2.drawContours(binImgColor, ContoursSeries[i], -1, (0,255,0), 0)
            if not np.isnan(xCents[i]):
                binImgColor = cv2.circle(binImgColor, (int(xCents[i]), int(yCents[i])), 5, (0,255,0), -1)
                binImgColor = cv2.ellipse(binImgColor, (int(xCents[i]), int(yCents[i])), (int(aVals[i]), int(bVals[i])), thetaVals[i] * 180 / np.pi, 0, 360, (255, 0, 0), 2)
                if aVals[i] >= bVals[i]:
                    binImgColor = cv2.line(binImgColor, (int(xCents[i] + aVals[i] * np.cos(thetaVals[i])), int(yCents[i] + aVals[i] * np.sin(thetaVals[i]))), (int(xCents[i] - aVals[i] * np.cos(thetaVals[i])), int(yCents[i] - aVals[i] * np.sin(thetaVals[i]))), (0, 0, 255), 2)
                else:
                    thetaValsNew = fixThetas([aVals[i]], [bVals[i]], [thetaVals[i]])
                    binImgColor = cv2.line(binImgColor, (int(xCents[i] + bVals[i] * np.cos(thetaValsNew[0])), int(yCents[i] + bVals[i] * np.sin(thetaValsNew[0]))), (int(xCents[i] - bVals[i] * np.cos(thetaValsNew[0])), int(yCents[i] - bVals[i] * np.sin(thetaValsNew[0]))), (0, 0, 255), 2)
                # print(aVals[i], bVals[i], thetaVals[i])
            combined = np.concatenate((frame, binImgColor), axis = 1)
#             writer.write(frame)
            cv2.imshow('frame',combined)
            
            # Move to next values in list
            i += 1
            
            if cv2.waitKey(50) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()
#     writer.release()
    cv2.waitKey(1)
    
    return

## Visualize all Boundary Positions

In [ ]:
def allBoundaries(video, xValsAll, yValsAll):
    fig = plt.figure(figsize=(6, 6))
    size = len(xValsAll)
    cmap = plt.get_cmap('rainbow')
    for i in range(len(xValsAll)):
        # Change to rainbow color
        plt.plot(xValsAll[i], yValsAll[i], ls='-', color = cmap(i / size))
        plt.title(video[:-4])
        plt.axis('off')
        plt.savefig("Tumbling/Figures/" + video[:-4] + ".png", dpi = 300)
    return 

## Main Function

In [ ]:
def mainFunction(path, videos):
    blur = 13
    avLinVels = []
    avAngVels = []
    for video in videos:
        if '.avi' in video and 'Modified' not in video:
            print(video)
            filename = path + '/' + video
            ContoursSeries = extractBoundaries(filename, blur)
            xCents, yCents, aVals, bVals, thetaVals, xValsAll, yValsAll = ellipseFit(ContoursSeries)
            fixedThetas = fixThetas(aVals, bVals, thetaVals)
            LinVel = avLinVel(xCents, yCents, 5)
            AngVel = avAngularVel(fixedThetas, 5)
            avLinVels.append(LinVel)
            avAngVels.append(AngVel)
            visualizeEllipse(filename, ContoursSeries, xCents, yCents, aVals, bVals, thetaVals)
            allBoundaries(video, xValsAll, yValsAll)
    return avLinVels, avAngVels

## Project Data

### Chondrogenesis SG and CG Day 0 

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
avLinVelsSG_D0_Chondro, avAngVelsSG_D0_Chondro = mainFunction(path, images)

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
avLinVelsCG_D0_Chondro, avAngVelsCG_D0_Chondro = mainFunction(path, images)

### Chondrogenesis SG Day 4

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
avLinVelsSG_D4_Chondro, avAngVelsSG_D4_Chondro = mainFunction(path, images)

### Adipogenesis SG and CG Day 0

In [ ]:
# SG
path = ""
images = os.listdir(path)
images.sort()
avLinVelsSG_D0_Adipo, avAngVelsSG_D0_Adipo = mainFunction(path, images)

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
avLinVelsCG_D0_Adipo, avAngVelsCG_D0_Adipo = mainFunction(path, images)

### Degradable CG Day 0 

In [ ]:
# CG
path = ""
images = os.listdir(path)
images.sort()
avLinVelsCG_D0_Deg, avAngVelsCG_D0_Deg = mainFunction(path, images)

### Combine All Data

In [ ]:
# Chondro SG Day 0 
data = {'avLinVelsSG_D0_Chondro': avLinVelsSG_D0_Chondro, 'avAngVelsSG_D0_Chondro': avAngVelsSG_D0_Chondro}
df_SG_D0_Chondro = pd.DataFrame(data)
# Chondro CG Day 0 
data = {'avLinVelsCG_D0_Chondro': avLinVelsCG_D0_Chondro, 'avAngVelsCG_D0_Chondro': avAngVelsCG_D0_Chondro}
df_CG_D0_Chondro = pd.DataFrame(data)
# Chondro SG Day 4
data = {'avLinVelsSG_D4_Chondro': avLinVelsSG_D4_Chondro, 'avAngVelsSG_D4_Chondro': avAngVelsSG_D4_Chondro}
df_SG_D4_Chondro = pd.DataFrame(data)
# Adipo SG Day 0
data = {'avLinVelsSG_D0_Adipo': avLinVelsSG_D0_Adipo, 'avAngVelsSG_D0_Adipo': avAngVelsSG_D0_Adipo}
df_SG_D0_Adipo = pd.DataFrame(data)
# Adipo CG Day 0
data = {'avLinVelsCG_D0_Adipo': avLinVelsCG_D0_Adipo, 'avAngVelsCG_D0_Adipo': avAngVelsCG_D0_Adipo}
df_CG_D0_Adipo = pd.DataFrame(data)
# Deg CG Day 0
data = {'avLinVelsCG_D0_Deg': avLinVelsCG_D0_Deg, 'avAngVelsCG_D0_Deg': avAngVelsCG_D0_Deg}
df_CG_D0_Deg = pd.DataFrame(data)

# Combine All data
allData = pd.concat([df_SG_D0_Chondro, df_CG_D0_Chondro, df_SG_D4_Chondro, df_SG_D0_Adipo, df_CG_D0_Adipo, df_CG_D0_Deg], axis=1)
allData.to_csv("Tumbling/Final Data/allData.csv")

In [ ]:
allData